In [82]:
#soo jiido Libarary
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix)

In [83]:
# soo jiido fileka kadibna check garee
df= pd.read_csv("mail_l7_dataset.csv")
print(df.head())
print(df.shape)
print(df.info)
print(df.describe())
print(df.isnull().sum())

  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
(5572, 2)
<bound method DataFrame.info of      Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5

In [84]:
# Data cleaning NaN ka saar
df = df.where(pd.notnull(df), "")
RANDOM_STATE = 42 
#isticmal lebelEncoding
df.loc[df["Category"].str.lower().str.strip() == "spam", "Category"] = 0
df.loc[df["Category"].str.lower().str.strip() == "ham", "Category"] = 1
print(df.head())

  Category                                            Message
0        1  Go until jurong point, crazy.. Available only ...
1        1                      Ok lar... Joking wif u oni...
2        0  Free entry in 2 a wkly comp to win FA Cup fina...
3        1  U dun say so early hor... U c already then say...
4        1  Nah I don't think he goes to usf, he lives aro...


In [85]:
#input x, y waa lebel
x = df["Message"].astype(str)
y = df["Category"].astype(int)


In [86]:
# u kala qaad training 80% iyo test 20%
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size=0.2, random_state=RANDOM_STATE)
#eeg train iyo test
print("Train", x_train.shape[0],"Test", x_test.shape[0])

Train 4457 Test 1115


In [87]:
# u badal text to numbers  tf.idf hight frequence sii ku wa soo laablabta sida win,free,click
tfidf = TfidfVectorizer(min_df=1, stop_words="english", lowercase=True) 

In [88]:
# qoraalka numbers ku bedel 
x_train_features = tfidf.fit_transform(x_train)
x_test_features = tfidf.transform(x_test)
print("x_train:", x_train_features[0], "x_test:", x_test_features[0])

x_train: <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14 stored elements and shape (1, 7440)>
  Coords	Values
  (0, 5512)	0.1898892037332199
  (0, 7222)	0.2173884735352799
  (0, 258)	0.2379428657041507
  (0, 7162)	0.2550284465664535
  (0, 354)	0.3544175987866074
  (0, 2724)	0.3544175987866074
  (0, 7300)	0.24288153842988894
  (0, 2049)	0.3034375179183143
  (0, 3262)	0.33791755486732394
  (0, 5800)	0.17558937755823417
  (0, 6264)	0.1898892037332199
  (0, 694)	0.3171299579602537
  (0, 2497)	0.2442158912653505
  (0, 5818)	0.22682143517864364 x_test: <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8 stored elements and shape (1, 7440)>
  Coords	Values
  (0, 1751)	0.2863401438180078
  (0, 2065)	0.296327528007439
  (0, 2173)	0.2473614085298311
  (0, 3225)	0.3299996466854455
  (0, 3395)	0.659999293370891
  (0, 3955)	0.3096991339243786
  (0, 4100)	0.27836537155821267
  (0, 4942)	0.22607959672331038


In [89]:
# training samee logistic regressin
lr = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE) 
lr.fit(x_train_features, y_train )
lr_predict = lr.predict(x_test_features)

In [90]:
#training random forest samee
rf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
rf.fit(x_train_features, y_train)
rf_predict = rf.predict(x_test_features.toarray())

In [91]:
# tarining samee Naive Bayes
NB = MultinomialNB()
NB.fit(x_train_features, y_train)
NB_predict = NB.predict(x_test_features)

In [92]:
# function ku so saaraya metrics 
def metrics (name, y_true, y_pred, pos_label=0):   #0 spam 
  Acc = accuracy_score(y_true, y_pred)                            
  Prec = precision_score(y_true, y_pred, pos_label=pos_label)
  Rec = recall_score(y_true, y_pred, pos_label=pos_label)
  F1 = f1_score(y_true, y_pred, pos_label=pos_label)
  print(f"{name} Performance")
  print(f"Accurancy: {Acc:.3f}")                           #overall negative,positive
  print(f"Precision: {Prec:.3f}(positive = spam = 0)")    #meequ si sax ah ugu tilmaamay spam
  print(f"Recall: {Rec:.3f}(positive = spam = 0)")        #overall spam intu ku tilmaamay
  print(f"F1-score: {F1:.3f}(positive = spam = 0)")    

In [93]:
# function ku so saaraya confusion_matrix
def conf_matrix(name, y_true, y_pred):
  cm = confusion_matrix(y_true, y_pred, labels=[1,0])
  cm_df = pd.DataFrame(
    cm,
    index= ["Actual Ham (1)", "Actual spam (0)"],
    columns=["Pred Ham (1)", "Pred spam (0)"]
  )
  print(f"{name} confution_matrix:\n{cm_df}")

In [94]:
# function ka soo eeg natiijada 3 Model logistic, forest, Navy Bais
metrics("LogisticRegression", y_test, lr_predict)
conf_matrix("LogisticRegression", y_test, lr_predict)

metrics("RandomForest", y_test, rf_predict)
conf_matrix("RandomForest", y_test, rf_predict)

metrics("Naive Bayes", y_test, NB_predict)
conf_matrix("Naive Bayes", y_test, NB_predict)

LogisticRegression Performance
Accurancy: 0.968
Precision: 1.000(positive = spam = 0)
Recall: 0.758(positive = spam = 0)
F1-score: 0.863(positive = spam = 0)
LogisticRegression confution_matrix:
                 Pred Ham (1)  Pred spam (0)
Actual Ham (1)            966              0
Actual spam (0)            36            113
RandomForest Performance
Accurancy: 0.983
Precision: 1.000(positive = spam = 0)
Recall: 0.872(positive = spam = 0)
F1-score: 0.932(positive = spam = 0)
RandomForest confution_matrix:
                 Pred Ham (1)  Pred spam (0)
Actual Ham (1)            966              0
Actual spam (0)            19            130
Naive Bayes Performance
Accurancy: 0.977
Precision: 1.000(positive = spam = 0)
Recall: 0.826(positive = spam = 0)
F1-score: 0.904(positive = spam = 0)
Naive Bayes confution_matrix:
                 Pred Ham (1)  Pred spam (0)
Actual Ham (1)            966              0
Actual spam (0)            26            123


In [95]:
#sanit check  single message  
i = 14     #sample row ga 14 aad weye
sample_text = x_test.iloc[i]    #xtest waa qoralkaas x feauture inputka
true_label = y_test.iloc[i]    #ham truelabel u baasaa

lr_pred_one = int(lr.predict(tfidf.transform([sample_text]))[0])
rf_pred_one = int(rf.predict(tfidf.transform([sample_text]).toarray())[0])

def lab2str(mohamed):
  return "spam(0)" if mohamed == 0  else "Ham (1)"

print("sanity Check single message")
snippet = (sample_text[:160] + "...") if len(sample_text) > 160 else sample_text
print("text snippet", snippet)
print("actual:", lab2str(true_label))
print("LR predict:", lab2str(lr_pred_one))
print("Rf predict:", lab2str(rf_pred_one))

sanity Check single message
text snippet FREE RINGTONE text FIRST to 87131 for a poly or text GET to 87131 for a true tone! Help? 0845 2814032 16 after 1st free, tones are 3x£150pw to e£nd txt stop
actual: spam(0)
LR predict: spam(0)
Rf predict: spam(0)


In [96]:
#sanit check  single message  
i = 102     
sample_text = x_test.iloc[i]    
true_label = y_test.iloc[i]   

lr_pred_one = int(lr.predict(tfidf.transform([sample_text]))[0])
rf_pred_one = int(rf.predict(tfidf.transform([sample_text]).toarray())[0])

def lab2str(mohamed):
  return "spam(0)" if mohamed == 0  else "Ham (1)"

print("sanity Check single message")
snippet = (sample_text[:160] + "...") if len(sample_text) > 160 else sample_text
print("text snippet", snippet)
print("actual:", lab2str(true_label))
print("LR predict:", lab2str(lr_pred_one))
print("Rf predict:", lab2str(rf_pred_one))

sanity Check single message
text snippet Dear got train and seat mine lower seat
actual: Ham (1)
LR predict: Ham (1)
Rf predict: Ham (1)


In [97]:
#sanit check  single message  
i = 539
sample_text = x_test.iloc[i]    
true_label = y_test.iloc[i]   

lr_pred_one = int(lr.predict(tfidf.transform([sample_text]))[0])
rf_pred_one = int(rf.predict(tfidf.transform([sample_text]).toarray())[0])

def lab2str(mohamed):
  return "spam(0)" if mohamed == 0  else "Ham (1)"

print("sanity Check single message")
snippet = (sample_text[:160] + "...") if len(sample_text) > 160 else sample_text
print("text snippet", snippet)
print("actual:", lab2str(true_label))
print("LR predict:", lab2str(lr_pred_one))
print("Rf predict:", lab2str(rf_pred_one))

sanity Check single message
text snippet PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S.I.M. points. Call 08718738001 Identifier Code: 49557 Expires 26/11/04
actual: spam(0)
LR predict: spam(0)
Rf predict: spam(0)


In [99]:
# Single-message sanity check sample prediction
samples = [
    "Ku guuleyso $10000 credit card. Claim now! guryo free ah hel gaari free ah ",
    "Congratulations ku shubo 100 dollarclick dheh oo hel bishan 200 dollar free ah Claim now!",
    "Congratulations, you won a free ticket",
    "You have been selected for a $1000 gift card. Claim now!",
    "FIRST to 87131 for a poly or text GET to 87131 for a true tone! Help? 0845 2814032 16 after 1st free, tones are 3x£150pw to e£nd txt stop",
    "Dear got train and seat mine lower seat",
    "Free entry in 2 a weekly competition!",
    "I will meet you at the cafe tomorrow",
    "Your 2003 Account Statement for shows 800 un-redeemed S.I.M. points. Call 08718738001 Identifier Code: 49557 Expires 26/11/04",
    "click badan samee si aan winner 1 u noqoto adna u hesho free credit card $10000 doller kadibna claim now."
]
def labstr(mphamed):  # readable output
    return "Spam (0)" if mphamed == 0 else "Ham (1)"

print("\n simple massege sample prediction")
for text in samples:
    print("text snippet", text)
    print("Lr Predict: ", labstr(int(lr.predict(tfidf.transform([text]))[0])))
    print("Rf Predict: ", labstr(int(rf.predict(tfidf.transform([text]).toarray())[0])))
    print("NB Predict: ", lab2str(int(NB.predict(tfidf.transform([text]))[0])))

    
    


 simple massege sample prediction
text snippet Ku guuleyso $10000 credit card. Claim now! guryo free ah hel gaari free ah 
Lr Predict:  Ham (1)
Rf Predict:  Ham (1)
NB Predict:  Ham (1)
text snippet Congratulations ku shubo 100 dollarclick dheh oo hel bishan 200 dollar free ah Claim now!
Lr Predict:  Spam (0)
Rf Predict:  Spam (0)
NB Predict:  spam(0)
text snippet Congratulations, you won a free ticket
Lr Predict:  Ham (1)
Rf Predict:  Ham (1)
NB Predict:  Ham (1)
text snippet You have been selected for a $1000 gift card. Claim now!
Lr Predict:  Spam (0)
Rf Predict:  Spam (0)
NB Predict:  spam(0)
text snippet FIRST to 87131 for a poly or text GET to 87131 for a true tone! Help? 0845 2814032 16 after 1st free, tones are 3x£150pw to e£nd txt stop
Lr Predict:  Spam (0)
Rf Predict:  Spam (0)
NB Predict:  spam(0)
text snippet Dear got train and seat mine lower seat
Lr Predict:  Ham (1)
Rf Predict:  Ham (1)
NB Predict:  Ham (1)
text snippet Free entry in 2 a weekly competition!
Lr Predict: 